> This is one of the 100 recipes of the [IPython Cookbook](http://ipython-books.github.io/), the definitive guide to high-performance scientific computing and data science in Python.


# 1.4. Creating an IPython extension with custom magic commands

1. Let's import a few functions from the IPython magic system.

In [1]:
from IPython.core.magic import (register_line_magic, 
                                register_cell_magic)

2. Defining a new line magic is particularly simple. First, let's create a function that accepts the contents of the line (except the initial `%`-prefixed magic command). The name of this function is the name of the magic. Then, let's decorate this function with `@register_line_magic`. We're done!

In [2]:
@register_line_magic
def hello(line):
    if line == 'french':
        print("Salut tout le monde!")
    else:
        print("Hello world!")

In [3]:
%hello

Hello world!


In [4]:
%hello french

Salut tout le monde!


3. Let's create a slightly more useful cell magic `%%csv` that parses a CSV string and returns a Pandas DataFrame object. This time, the function takes as argument the first line (what follows `%%csv`), and the contents of the cell (everything in the cell except the first line).

In [5]:
import pandas as pd
#from StringIO import StringIO  # Python 2
from io import StringIO  # Python 3

@register_cell_magic
def csv(line, cell):
    # We create a string buffer containing the
    # contents of the cell.
    sio = StringIO(cell)
    # We use Pandas' read_csv function to parse
    # the CSV string.
    return pd.read_csv(sio)

In [6]:
%%csv
col1,col2,col3
0,1,2
3,4,5
7,8,9

,col1,col2,col3
0,0,1,2
1,3,4,5
2,7,8,9


We can access the returned object with `_`.

In [7]:
df = _
df.describe()

,col1,col2,col3
count,3.000000,3.000000,3.000000
mean,3.333333,4.333333,5.333333
std,3.511885,3.511885,3.511885
min,0.000000,1.000000,2.000000
25%,1.500000,2.500000,3.500000
50%,3.000000,4.000000,5.000000
75%,5.000000,6.000000,7.000000
max,7.000000,8.000000,9.000000


4. The method we described is useful in an interactive session. If you want to use the same magic in multiple notebooks, or if you want to distribute it, you need to create an **IPython extension** that implements your custom magic command. Let's show how to do that. The first step is to create a Python script (`csvmagic.py` here) that implements the magic.

In [16]:
%%writefile ~/.ipython/extensions/csvmagic.py
import pandas as pd
#from StringIO import StringIO  # Python 2
from io import StringIO  # Python 3

def csv(line, cell):
    sio = StringIO(cell)
    return pd.read_csv(sio)

def load_ipython_extension(ipython):
    """This function is called when the extension is loaded.
    It accepts an IPython InteractiveShell instance.
    We can register the magic with the `register_magic_function`
    method of the shell instance."""
    ipython.register_magic_function(csv, 'cell')

Writing /Users/sohei/.ipython/extensions/csvmagic.py


5. Once the extension is created, we need to import it in the IPython session. The `%load_ext` magic command takes the name of a Python module and imports it, calling immediately `load_ipython_extension`. Here, loading this extension automatically registers our magic function `%%csv`. The Python module needs to be importable. Here, it is in the current directory. In other situations, it has to be in the Python path. It can also be stored in `~\.ipython\extensions` which is automatically put in the Python path.

In [9]:
%load_ext csvmagic

In [10]:
%%csv
col1,col2,col3
0,1,2
3,4,5
7,8,9

,col1,col2,col3
0,0,1,2
1,3,4,5
2,7,8,9


Finally, to ensure that this magic is automatically defined in our IPython profile, we can instruct IPython to load this extension at startup. To do this, let's open the file `~/.ipython/profile_default/ipython_config.py` and let's put `'csvmagic'` in the `c.InteractiveShellApp.extensions` list. The `csvmagic` module needs to be importable. It is common to create a *Python package* implementing an IPython extension, which itself defines custom magic commands.

> You'll find all the explanations, figures, references, and much more in the book (to be released later this summer).

> [IPython Cookbook](http://ipython-books.github.io/), by [Cyrille Rossant](http://cyrille.rossant.net), Packt Publishing, 2014 (500 pages).

In [12]:
shell = get_ipython()

In [13]:
shell.user_ns

{'In': ['',
  'from IPython.core.magic import (register_line_magic, \n                                register_cell_magic)',
  '@register_line_magic\ndef hello(line):\n    if line == \'french\':\n        print("Salut tout le monde!")\n    else:\n        print("Hello world!")',
  "get_ipython().magic('hello')",
  "get_ipython().magic('hello french')",
  "import pandas as pd\n#from StringIO import StringIO  # Python 2\nfrom io import StringIO  # Python 3\n\n@register_cell_magic\ndef csv(line, cell):\n    # We create a string buffer containing the\n    # contents of the cell.\n    sio = StringIO(cell)\n    # We use Pandas' read_csv function to parse\n    # the CSV string.\n    return pd.read_csv(sio)",
  "get_ipython().run_cell_magic('csv', '', 'col1,col2,col3\\n0,1,2\\n3,4,5\\n7,8,9')",
  'df = _\ndf.describe()',
  'get_ipython().run_cell_magic(\'writefile\', \'csvmagic.py\', \'import pandas as pd\\n#from StringIO import StringIO  # Python 2\\nfrom io import StringIO  # Python 3\\n\\ndef